STT를 위한 whisper모델을 미리 다운을 받고, 실행시키기 위해 빌드합니다.  
GPU 쓰고싶으면 런타임구성에서 T4쓰시면 됩니당

In [1]:
# 📦 필수 패키지 설치
!apt-get install -y build-essential cmake ffmpeg

# 📥 whisper.cpp 클론
!git clone https://github.com/ggerganov/whisper.cpp.git

# 📁 디렉토리 이동
%cd whisper.cpp

# # 🔨 CMake로 빌드 M4 맥북에서 CPU로 돌려볼려면 이거 써주세용
# !cmake -B build
# !cmake --build build --config Release

# 🔨 CUDA 활성화 빌드 # 코랩으로 할려니 너무오래걸려서 잠시 GPU 쓰겠음
# M4 맥북 [stt시간 4분? ]
# Colab CPU - 너무오래걸림
!cmake -B build -DGGML_CUDA=1
!cmake --build build --config Release

# 다시 디렉토리 밖으로 이동
%cd ..

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Cloning into 'whisper.cpp'...
remote: Enumerating objects: 17408, done.
remote: Counting objects: 100% (459/459), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 17408 (delta 310), reused 246 (delta 246), pack-reused 16949 (from 4)
Receiving objects: 100% (17408/17408), 20.63 MiB | 7.95 MiB/s, done.
Resolving deltas: 100% (11952/11952), done.
/content/whisper.cpp
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the pr

영상 전처리를 위한 ffmpeg을 다운로드합니다.

In [4]:
# gpu 세팅 잘되있나 확인
!nvidia-smi

Sat Apr 19 18:49:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              8W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
 # 기존 ffmpeg 제거 (선택)
!apt-get remove ffmpeg -y

# 최신 ffmpeg 바이너리 다운로드 (from John Van Sickle build)
!wget -O ffmpeg-release-amd64-static.tar.xz https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz

# 압축 해제
!tar -xf ffmpeg-release-amd64-static.tar.xz

# ffmpeg 폴더 이름 찾기 (보통 ffmpeg-6.0-... 식)
import os
ffmpeg_dir = [d for d in os.listdir() if d.startswith("ffmpeg") and os.path.isdir(d)][0]
print("📦 ffmpeg dir:", ffmpeg_dir)

# 심볼릭 링크로 경로 등록
!cp ./{ffmpeg_dir}/ffmpeg /usr/local/bin/ffmpeg
!cp ./{ffmpeg_dir}/ffprobe /usr/local/bin/ffprobe

# 확인
!ffmpeg -version

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'ffmpeg' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
--2025-04-19 18:49:24--  https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz
Resolving johnvansickle.com (johnvansickle.com)... 107.180.57.212
Connecting to johnvansickle.com (johnvansickle.com)|107.180.57.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41888096 (40M) [application/x-xz]
Saving to: ‘ffmpeg-release-amd64-static.tar.xz’

ffmpeg-release-amd6 100%[===================>]  39.95M  14.8MB/s    in 2.7s    

2025-04-19 18:49:28 (14.8 MB/s) - ‘ffmpeg-release-amd64-static.tar.xz’ saved [41888096/41888096]

📦 ffmpeg dir: ffmpeg-7.0.2-amd64-static
ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
built with gcc 8 (Debian 8.3.0-6)
configuration: --enable-gpl --enabl

In [6]:
!pip install huggingface_hub
!pip install -U langchain
!pip install -U langchain-community
!pip install -U langchain-openai
!pip install -U faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 21.0 MB/s eta 0:00:00


In [7]:
import os
import json
import gdown
import shutil
import subprocess
from getpass import getpass
from langchain.chains import RetrievalQA
from huggingface_hub import hf_hub_download
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

whisper 모델중 ggml-small 모델을 사용합니다.
tiny는 조금 빠르지만, 성능이 안좋고,,, small이 그나마 성능좋고 빠른정도?

In [11]:
# 파일 이름 설정
LECTURE_FILE_NAME = "01"
MODEL_NAME = "ggml-small.bin"

os.makedirs("./AX_Data", exist_ok=True)
os.makedirs(f"./Sound/{LECTURE_FILE_NAME}", exist_ok=True)
os.makedirs(f"./Text/{LECTURE_FILE_NAME}", exist_ok=True)

# 🔽 모델 다운로드
MODEL_PATH = hf_hub_download(
    repo_id="ggerganov/whisper.cpp",
    filename=MODEL_NAME
)
print(f"모델 다운로드 완료 : {MODEL_PATH}")

모델 다운로드 완료 : /root/.cache/huggingface/hub/models--ggerganov--whisper.cpp/snapshots/5359861c739e955e79d9a303bcbc70fb988958b1/ggml-small.bin


선임님의 강의영상을 다운로드합니다.

In [12]:
video_path = f"./AX_Data/{LECTURE_FILE_NAME}.mp4"
# Google Drive에서 영상 다운로드
file_id = "1h9lxMPkahHB-dn7p4QBXkMwOZpgVaoSB"  # Google Drive 공유 링크에서 file_id 추출
url = f"https://drive.google.com/uc?id={file_id}"

# 다운로드
gdown.download(url, video_path, quiet=False)

print(f"✅ 영상 다운로드 완료: {video_path}")

Downloading...
From (original): https://drive.google.com/uc?id=1h9lxMPkahHB-dn7p4QBXkMwOZpgVaoSB
From (redirected): https://drive.google.com/uc?id=1h9lxMPkahHB-dn7p4QBXkMwOZpgVaoSB&confirm=t&uuid=dbd7b2ee-287c-4bf1-8360-b3f5a9c6cee2
To: /content/AX_Data/01.mp4
100%|██████████| 495M/495M [00:07<00:00, 62.0MB/s]

✅ 영상 다운로드 완료: ./AX_Data/01.mp4


ffmpeg을 이용하여 영상-> 소리를 추출합니다.   
다만 4시간을 그대로 소리로 추출하기에 너무 용량이 크기에 silent 처리되는 소리 (영상중간의 무음)  
은 제거하여 83분 분량의 길이로 줄입니다.  

여기에서 2분단위로 자르는 이유는 whisper 모델이 auto-regressive 구조기에
긴 소리의 경우, 동일한 대답만을 반복하는 에러를 return 하기에 2분단위로 잘라서 ffmpeg으로 저장합니다.

추가로 m4 맥북에서는 4분정도밖에 안걸리는데 colab환경에서는 오래걸림.

In [13]:
chunk_output_dir = f"./Sound/{LECTURE_FILE_NAME}"

os.system(
    f"ffmpeg -y -i {video_path} "
    f"-vn -ar 16000 -ac 1 "
    f"-af silenceremove=start_periods=1:start_duration=1:start_threshold=-45dB:"
    f"stop_periods=-1:stop_duration=1:stop_threshold=-45dB,"
    f"aresample=async=1 "
    f"-f segment -segment_time 120 -c:a pcm_s16le "
    f"{chunk_output_dir}/chunk_%03d.wav"
)

print("✅ 무음 제거 + 2분 자르기 한 번에 완료")

✅ 무음 제거 + 2분 자르기 한 번에 완료


STT를 진행하기 위한 함수를 작성합니다.

In [14]:
def whisper_cpp_transcribe(file_name, model_name):
    model_label = model_name.replace(".bin", "")
    audio_path = f"./Sound/{LECTURE_FILE_NAME}/{file_name}.wav"
    output_file_prefix = f"./Text/{LECTURE_FILE_NAME}/{file_name}_{model_label}"

    os.makedirs(f"./Text/{LECTURE_FILE_NAME}", exist_ok=True)

    cmd = [
        "./whisper.cpp/build/bin/whisper-cli",
        "-m", MODEL_PATH,
        "-f", audio_path,
        "-l", "ko",
        "-oj",
        "-of", output_file_prefix,
    ]

    subprocess.run(cmd)
    print(f"✅ {file_name} 처리 완료 → {output_file_prefix}.json")

In [15]:
# ✅ .wav 파일만 필터링
sound_dir = f"./Sound/{LECTURE_FILE_NAME}"
chunk_files = sorted([
    f for f in os.listdir(sound_dir)
    if f.endswith(".wav")
])

chunk_count = len(chunk_files)
print("📦 chunk count:", chunk_count)
print("🎵 chunk files:", chunk_files[:5], "...")

# ✅ 병합용 경로 설정
model_label = MODEL_NAME.replace(".bin", "")
output_merged_path = f"./Text/{LECTURE_FILE_NAME}/{LECTURE_FILE_NAME}_merged_{model_label}.json"
merged_segments = []

# ✅ 42개의 chunk 처리
for i in range(chunk_count):
    chunk_id = f"chunk_{i:03d}"

    # whisper 처리 (여기 함수는 기존 그대로 사용)
    whisper_cpp_transcribe(chunk_id, MODEL_NAME)

    # 결과 JSON 읽기
    json_path = f"./Text/{LECTURE_FILE_NAME}/{chunk_id}_{model_label}.json"
    if os.path.exists(json_path):
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            merged_segments.extend(data.get("transcription", []))

# ✅ 최종 병합된 JSON 저장
with open(output_merged_path, "w", encoding="utf-8") as f:
    json.dump({"transcription": merged_segments}, f, indent=2, ensure_ascii=False)

print(f"✅ 전체 병합 완료 → {output_merged_path}")

📦 chunk count: 42
🎵 chunk files: ['chunk_000.wav', 'chunk_001.wav', 'chunk_002.wav', 'chunk_003.wav', 'chunk_004.wav'] ...
✅ chunk_000 처리 완료 → ./Text/chunk_000_ggml-small.json
✅ chunk_001 처리 완료 → ./Text/chunk_001_ggml-small.json
✅ chunk_002 처리 완료 → ./Text/chunk_002_ggml-small.json
✅ chunk_003 처리 완료 → ./Text/chunk_003_ggml-small.json
✅ chunk_004 처리 완료 → ./Text/chunk_004_ggml-small.json
✅ chunk_005 처리 완료 → ./Text/chunk_005_ggml-small.json
✅ chunk_006 처리 완료 → ./Text/chunk_006_ggml-small.json
✅ chunk_007 처리 완료 → ./Text/chunk_007_ggml-small.json
✅ chunk_008 처리 완료 → ./Text/chunk_008_ggml-small.json
✅ chunk_009 처리 완료 → ./Text/chunk_009_ggml-small.json
✅ chunk_010 처리 완료 → ./Text/chunk_010_ggml-small.json
✅ chunk_011 처리 완료 → ./Text/chunk_011_ggml-small.json
✅ chunk_012 처리 완료 → ./Text/chunk_012_ggml-small.json
✅ chunk_013 처리 완료 → ./Text/chunk_013_ggml-small.json
✅ chunk_014 처리 완료 → ./Text/chunk_014_ggml-small.json
✅ chunk_015 처리 완료 → ./Text/chunk_015_ggml-small.json
✅ chunk_016 처리 완료 → ./Text/ch

In [23]:
# STEP 1. Whisper JSON 불러오기
with open(f"./Text/{LECTURE_FILE_NAME}/{LECTURE_FILE_NAME}_merged_ggml-small.json", "r", encoding="utf-8") as f:
    data = json.load(f)

segments = data["transcription"]
segments[:5]

[{'timestamps': {'from': '00:00:00,000', 'to': '00:00:20,980'},
  'offsets': {'from': 0, 'to': 20980},
  'text': ' (인터뷰)'},
 {'timestamps': {'from': '00:00:20,980', 'to': '00:00:25,580'},
  'offsets': {'from': 20980, 'to': 25580},
  'text': ' 혹시 여러분, 젠킨스라는 툴을 해본 적이 있다.'},
 {'timestamps': {'from': '00:00:25,580', 'to': '00:00:27,580'},
  'offsets': {'from': 25580, 'to': 27580},
  'text': ' 젠킨스 써본 적 있다.'},
 {'timestamps': {'from': '00:00:27,580', 'to': '00:00:32,400'},
  'offsets': {'from': 27580, 'to': 32400},
  'text': ' 써보실 때 어느 정도 쓰시니 까지 써보셨어요.'},
 {'timestamps': {'from': '00:00:32,400', 'to': '00:00:35,560'},
  'offsets': {'from': 32400, 'to': 35560},
  'text': ' 그냥 사용자 입장에서 파이프라인이 짜는 정도?'}]

In [25]:
# STEP 2. 전체 텍스트 이어붙이기
full_text = "\n".join([seg["text"] for seg in segments])
full_text[:100]

' (인터뷰)\n 혹시 여러분, 젠킨스라는 툴을 해본 적이 있다.\n 젠킨스 써본 적 있다.\n 써보실 때 어느 정도 쓰시니 까지 써보셨어요.\n 그냥 사용자 입장에서 파이프라인이 짜는 정'

1000개의 청크 단위로, document를 나눕니다. 나누어진 document는
하나당 각 벡터공간으로 embedding 됩니다.

이때 1000개 단위로 50개씩 겹쳐서 document를 만듭니다.

In [26]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50
)

chunks = text_splitter.split_text(full_text)


In [27]:
os.environ["OPENAI_API_KEY"] = getpass("🔐 OpenAI API Key를 입력하세요: ")

🔐 OpenAI API Key를 입력하세요: ··········


인 메모리 vector database인 FAISS에 임베딩합니다.  
(나누어진 도큐먼트들)

In [28]:
# STEP 4. Document 리스트로 변환
docs = [Document(page_content=chunk, metadata={}) for chunk in chunks]

# STEP 5. Embedding + Vectorstore 생성
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding)

MMR(maximal marginal relevance) 알고리즘을 사용하여
중복되는 document를 return하지 않도록 합니다.

100개중에 중복되는것들을 다지우고 10개를 남깁니다.

추가로 Langchain 대답 return 방식은 Stuffing 방식을 사용합니다.

In [29]:
# STEP 6. Retriever + QA Chain 구성
retriever = vectorstore.as_retriever(search_type="mmr")
retriever.search_kwargs.update({
    "k": 10,
    "fetch_k": 100,
    "maximal_marginal_relevance": True
})

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

<ipython-input-29-9902a6564352>:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(temperature=0),


테스트를 한번해봅니다..

In [31]:
# STEP 7. 유저 질문 테스트
query = "강사님이 강의 전반적으로 강조하시는 부분이 뭐야?"
result = qa(query)

print("💬 Question:", query)
print("🧠 Answer:", result['result'])
print("📄 Source Metadata:", result['source_documents'])

💬 Question: 강사님이 강의 전반적으로 강조하시는 부분이 뭐야?
🧠 Answer: 강사님은 젠킨스와 관련된 주제를 중점적으로 다루고 계시며, 젠킨스의 기능과 활용 방법에 대해 설명하고 있습니다. 또한 오픈소스 툴과 플러그인을 활용하여 젠킨스를 보다 효율적으로 사용하는 방법에 대해 강조하고 계십니다. 또한 젠킨스를 통한 CI/CD 프로세스와 관련된 내용을 강조하고 있습니다.
📄 Source Metadata: [Document(id='53650263-abdf-4096-a9e1-dcd78c6ef9e2', metadata={}, page_content='- 형님, 이거 안 찍어 봐. - 형님, 이거 안 찍어 봐.\n - 형님, 이거 안 찍어 봐. - 형님, 이거 안 찍어 봐.\n - 형님, 이거 안 찍어 봐. - 형님, 이거 안 찍어 봐.\n - 형님, 이거 안 찍어 봐. - 형님, 이거 안 찍어 봐.\n - 형님, 이거 안 찍어 봐. - 형님, 이거 안 찍어 봐.\n - 형님, 이거 안 찍어 봐. - 형님, 이거 안 찍어 봐.\n - 형님, 이거 안 찍어 봐. - 형님, 이거 안 찍어 봐.\n 저희 올렸는데 거기서 왜 안 올리니깐 아까 부엉때리기 방함이\n 7박 4일에 3박 4일 기여서 7박 3일 같다고요?\n 그쵸 그쵸 그쵸 그정도 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸\n 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸 그쵸\n 조금씩 깔을 수 있는데\n 좀 더 이번엔 7번은 깔을 수 있도록 하겠습니다\n 한 번 더 깔을 수 있도록 하겠습니다\n 굉장히 많으니깐\n 7번 깔을 수 있도록 하겠습니다\n 한 번 다시 깔을 수 있도록 하겠습니다\n 다음은 어떻게 써볼까요?\n 진짜 마지막에 깔을 수 있어요\n 네, 마지막에 깔을 수 있을까요?\n 마지막에 깔을 수 있을까요?\n